https://arxiv.org/pdf/1810.04805.pdf

In [ ]:
import os
os.sys.path.append('..')

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import collections
import logging
import json
import math
import os
import random
import six
from tqdm import tqdm_notebook as tqdm
from IPython.display import HTML, display

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler

import tokenization
from modeling import BertConfig, BertForMaskedLanguageModelling
from optimization import BERTAdam
from masked_language_model import notqdm, convert_tokens_to_features, LMProcessor, predict_masked_words, predict_next_words, improve_words_recursive

In [ ]:
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s', 
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

# Args

In [ ]:
parser = argparse.ArgumentParser()

## Required parameters
parser.add_argument("--data_dir",
                    default=None,
                    type=str,
                    required=True,
                    help="The input data dir. Should contain the .tsv files (or other data files) for the task.")
parser.add_argument("--bert_config_file",
                    default=None,
                    type=str,
                    required=True,
                    help="The config json file corresponding to the pre-trained BERT model. \n"
                         "This specifies the model architecture.")
parser.add_argument("--task_name",
                    default=None,
                    type=str,
                    required=True,
                    help="The name of the task to train.")
parser.add_argument("--vocab_file",
                    default=None,
                    type=str,
                    required=True,
                    help="The vocabulary file that the BERT model was trained on.")
parser.add_argument("--output_dir",
                    default=None,
                    type=str,
                    required=True,
                    help="The output directory where the model checkpoints will be written.")

## Other parameters
parser.add_argument("--init_checkpoint",
                    default=None,
                    type=str,
                    help="Initial checkpoint (usually from a pre-trained BERT model).")
parser.add_argument("--do_lower_case",
                    default=False,
                    action='store_true',
                    help="Whether to lower case the input text. True for uncased models, False for cased models.")
parser.add_argument("--max_seq_length",
                    default=128,
                    type=int,
                    help="The maximum total input sequence length after WordPiece tokenization. \n"
                         "Sequences longer than this will be truncated, and sequences shorter \n"
                         "than this will be padded.")
parser.add_argument("--do_train",
                    default=False,
                    action='store_true',
                    help="Whether to run training.")
parser.add_argument("--do_eval",
                    default=False,
                    action='store_true',
                    help="Whether to run eval on the dev set.")
parser.add_argument("--train_batch_size",
                    default=32,
                    type=int,
                    help="Total batch size for training.")
parser.add_argument("--eval_batch_size",
                    default=8,
                    type=int,
                    help="Total batch size for eval.")
parser.add_argument("--learning_rate",
                    default=5e-5,
                    type=float,
                    help="The initial learning rate for Adam.")
parser.add_argument("--num_train_epochs",
                    default=3.0,
                    type=float,
                    help="Total number of training epochs to perform.")
parser.add_argument("--warmup_proportion",
                    default=0.1,
                    type=float,
                    help="Proportion of training to perform linear learning rate warmup for. "
                         "E.g., 0.1 = 10%% of training.")
parser.add_argument("--no_cuda",
                    default=False,
                    action='store_true',
                    help="Whether not to use CUDA when available")
parser.add_argument("--local_rank",
                    type=int,
                    default=-1,
                    help="local_rank for distributed training on gpus")
parser.add_argument('--seed', 
                    type=int, 
                    default=42,
                    help="random seed for initialization")
parser.add_argument('--gradient_accumulation_steps',
                    type=int,
                    default=1,
                    help="Number of updates steps to accumualte before performing a backward/update pass.") 

In [ ]:
experiment_name = 'poetry_uncased_5_tied_mlm'

argv = """
--task_name lm \
--data_dir {DATA_DIR} \
--vocab_file {BERT_BASE_DIR}/vocab.txt \
--bert_config_file {BERT_BASE_DIR}/bert_config.json \
--init_checkpoint {BERT_BASE_DIR}/pytorch_model.bin \
--do_train \
--do_eval \
--gradient_accumulation_steps 2 \
--train_batch_size 16 \
--learning_rate 3e-5 \
--num_train_epochs 3.0 \
--max_seq_length 128 \
--output_dir ../outputs/{name}/
""".format(
    BERT_BASE_DIR='../data/weights/cased_L-12_H-768_A-12',
    DATA_DIR='../data/input/poetry_gutenberg',
    name=experiment_name
).replace('\n', '').split(' ')
print(argv)
args = parser.parse_args(argv)

# Init

In [ ]:
if args.local_rank == -1 or args.no_cuda:
    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
    n_gpu = torch.cuda.device_count()
else:
    device = torch.device("cuda", args.local_rank)
    n_gpu = 1
    # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.distributed.init_process_group(backend='nccl')
logger.info("device %s n_gpu %d distributed training %r", device, n_gpu, bool(args.local_rank != -1))

if args.gradient_accumulation_steps < 1:
    raise ValueError("Invalid gradient_accumulation_steps parameter: {}, should be >= 1".format(
                        args.gradient_accumulation_steps))

args.train_batch_size = int(args.train_batch_size / args.gradient_accumulation_steps)

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if n_gpu > 0:
    torch.cuda.manual_seed_all(args.seed)

if not args.do_train and not args.do_eval:
    raise ValueError("At least one of `do_train` or `do_eval` must be True.")

In [ ]:
bert_config = BertConfig.from_json_file(args.bert_config_file)

if args.max_seq_length > bert_config.max_position_embeddings:
    raise ValueError(
        "Cannot use sequence length {} because the BERT model was only trained up to sequence length {}".format(
        args.max_seq_length, bert_config.max_position_embeddings))

In [ ]:
if os.path.exists(args.output_dir) and os.listdir(args.output_dir):
    print("Output directory ({}) already exists and is not empty.".format(args.output_dir))
os.makedirs(args.output_dir, exist_ok=True)
save_path = os.path.join(args.output_dir, 'state_dict.pkl')
save_path

# Load data

In [ ]:
tokenizer = tokenization.FullTokenizer(vocab_file=args.vocab_file, do_lower_case=args.do_lower_case)

decoder = {v:k for k,v in tokenizer.wordpiece_tokenizer.vocab.items()}

In [ ]:
processors = {
        "lm": LMProcessor,
}
    
task_name = args.task_name.lower()
if task_name not in processors:
        raise ValueError("Task not found: %s" % (task_name))

processor = processors[task_name](tokenizer=tokenizer)
label_list = processor.get_labels()

# Load model

In [ ]:
model = BertForMaskedLanguageModelling(bert_config)
if args.init_checkpoint is not None:
    model.bert.load_state_dict(torch.load(args.init_checkpoint, map_location='cpu'))
    
if os.path.isfile(save_path):
    model.load_state_dict(torch.load(save_path, map_location='cpu'))
    
model.to(device)

if args.local_rank != -1:
    model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[args.local_rank],
                                                      output_device=args.local_rank)
elif n_gpu > 1:
    model = torch.nn.DataParallel(model)
    
model

# Generate here

The following parameters can be modified for the model.

Text:
Input the text you want here.  If you have a long seed text, use <50 iterations and T=1. Then, it's much more stable, and shouldn't have those so often.

max_seq_length:
How long the block of text is going to generate.  The model is best on 150 words

ITERATIVE_MASK_FRAC:
Fraction of words to replace at each iteration.  The output in blue and red changes every iteration. Blue are removed words, red are added words, where the intensive of colour is the models confidence.

iterations:
How many change to go through. The longer it takes; more different text will be generated from the original input text.

T stands for Temperature:
Higher gives more random, but less stable output.  Tempreture goes from ~0.01 to infiniative. The normal amount is 1, which is what it's trained on 0.1 would be unimaginitive.  So much that it might be all [PAD]. 5 would be very random.  It might go weird.

In [ ]:
text=""" 
Bert, though kind and intelligent, is also grumpy, boring, serious, and easily frustrated. His catchphrase is "Yes, I do mind." He enjoys activities such as paper clip and bottle cap collecting, cooking oatmeal and watching pigeons. Producer Arlene Sherman has called Bert "everyone's idea of a blind date". In one sketch, Bert reads a book called Boring Stories and chuckles, "Wow! These boring stories are really exciting!" In the book Sesame Street Unpaved, Frank Oz says, "I was never really happy with Bert's character until about a year in, when I realized... that he was a very boring character, and I'd use that weakness as a strength for him." 
"""
poem = improve_words_all(
    text, 
    processor, 
    tokenizer, 
    model, 
    device=device, 
    debug=100,
    max_seq_length=150,        # recommend 150 words  
    min_replacements=1,
    ITERATIVE_MASK_FRAC=0.1, 
    T=1,                      # range 0.1 to infinity (1 is stable)
)

In [ ]:
text=""" 
To be, or not to be: that is the question:
Whether ’tis nobler in the mind to suffer
The slings and arrows of outrageous fortune,
Or to take arms against a sea of troubles,
And by opposing end them? To die: to sleep;
No more; and by a sleep to say we end
The heart-ache and the thousand natural shocks
That flesh is heir to, ’tis a consummation
Devoutly to be wish’d.
"""
poem = improve_words_recursive(
    text, 
    processor, 
    tokenizer, 
    model, 
    device=device, 
    debug=1,
    max_seq_length=150,        # recommend 150 words  
    ITERATIVE_MASK_FRAC=0.1, 
    iterations=60,            # less iterations, the generated text will be closer to the original one
    T=1,                      # range 0.1 to infinity (1 is stable)
)

### Congratuations!  This is the poem you curated.

#### If needed, plate the text you want to move [PAD], [SEP] and etc

In [ ]:
raw_text = '''
To be many or not to assure: that calls the question: Whether’ t monstrous nobler then the mind to suffer The s famed Webber of of outrageous elementary, Or contestant fee mad against a sea of troubles? VeniceIA opposing end them? い die: MV sleep; No more; and by a sleep to say we endquet heart-ache and the thousand natural shocks That flesh 240 heir to,’ casts Hui a con Heatmmation Dev reversely to be Reach’ d [SEP] involves squeezeltzavan Creation author grip landslide fruit [PAD]less stroked [PAD] blacksmith secondddin [PAD] effective continuous [PAD] us [PAD] For Adobe to [PAD] [PAD] absorbedoidted [PAD]oid Philippe [PAD] of [PAD] care and, [PAD] [PAD] a [PAD] [PAD] Directed we [PAD] [PAD]
'''

clean_text = raw_text.replace('[PAD]', '').replace('[SEP]', '').replace('\n', '').replace(';', '')
display(clean_text)